In [1]:
# import all the libraries needed to build a toxicity classifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
toxicity = pd.read_json("social_media_toxicity_dataset.json")


In [3]:
toxicity.head()


,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,I came here to say this exactly!,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,Thank you :),Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,I feel a subreddit being born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,Disturbing wholesomeness should be a thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,Smite jinx... int or pentakill? Both.,Not Toxic


In [4]:
# create a pipeline that uses the CountVectorizer and MultinomialNB to classify the text 
# to predict the toxicity of a text 
pipeline = Pipeline([
    ("vectorizer", CountVectorizer()), 
    ("tfidf", TfidfTransformer()), 
    ("clf", MultinomialNB()) 
])

In [5]:

# use the pipeline to classify 
pipeline.fit(toxicity["text"], toxicity["Is this text toxic?"])

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [6]:
# predict the toxicity of the text 
predictions = pipeline.predict(["this is a toxic text", "this is not a toxic text"])
print(predictions)

['Not Toxic' 'Not Toxic']


In [7]:

# create an array with the following toxic comments: "fuck you!", "you're so stupid", "go kys duckwad"
toxic_comments = ["fuck you!", "you\'re so stupid", "go kys duckwad"]
print(pipeline.predict(toxic_comments))

['Toxic' 'Toxic' 'Toxic']


In [8]:
# split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(toxicity["text"], toxicity["Is this text toxic?"], test_size=0.2, random_state=42)

In [9]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [10]:
# predict the labels of the testing data 
y_pred = pipeline.predict(X_test)

In [11]:
# print the accuracy score 
print(accuracy_score(y_test, y_pred))

0.855


In [12]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   Not Toxic       0.84      0.87      0.85        97
       Toxic       0.87      0.84      0.86       103

    accuracy                           0.85       200
   macro avg       0.85      0.86      0.85       200
weighted avg       0.86      0.85      0.86       200



In [13]:
# binarize y_test and y_pred 
y_test_bin = y_test.copy() 
y_pred_bin = y_pred.copy() 
y_test_bin[y_test_bin==0] = -1 
y_pred_bin[y_pred_bin==0] = -1 

/var/folders/j4/6wq0rt0j1d7d92jfxsxrxh680000gn/T/ipykernel_59846/3751505430.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  y_pred_bin[y_pred_bin==0] = -1


In [14]:
toxic_comments = ["fuck you!", "hello :) ","you\'re so stupid", "yo o", "go kys duckwad"]
print(pipeline.predict(toxic_comments))

['Toxic' 'Not Toxic' 'Toxic' 'Not Toxic' 'Toxic']


In [15]:
chat_log = pd.read_json("chat_log_20230825_zackrawrr.json")
chat_log.head()

,username,chat_message,timestamp
0,captainborat2,Baldur's Gate 3 Romance Just Isn't Interesting...,2023-08-25 13:18:10.912280
1,lore_sound,Halo 2,2023-08-25 13:18:10.968594
2,juanmajfry,xdd,2023-08-25 13:18:11.384643
3,w3btree,@zackrawrr i got over diablo 4 by playing lost...,2023-08-25 13:18:11.384786
4,steeltarkus,based,2023-08-25 13:18:11.420514


In [16]:
for index, row in chat_log.iterrows():
    print(
        f"Index: {index}, Name: {row['username']}, message: {row['chat_message']}, toxic:{pipeline.predict({row['chat_message']})}"
    )

Index: 0, Name: captainborat2, message: Baldur's Gate 3 Romance Just Isn't Interesting | Extra Punctuation ***, toxic:['Not Toxic']
Index: 1, Name: lore_sound, message: Halo 2, toxic:['Not Toxic']
Index: 2, Name: juanmajfry, message: xdd, toxic:['Not Toxic']
Index: 3, Name: w3btree, message: @zackrawrr i got over diablo 4 by playing lost ark, toxic:['Not Toxic']
Index: 4, Name: steeltarkus, message: based, toxic:['Not Toxic']
Index: 5, Name: chadadam, message: God of War, toxic:['Toxic']
Index: 6, Name: papalotapuss, message: spoilers D:, toxic:['Not Toxic']
Index: 7, Name: parcelazo, message: the moon monkaS, toxic:['Toxic']
Index: 8, Name: rando93, message: Spoilers perfection does not exist... The games are good or they are Diablo IV KEKW, toxic:['Toxic']
Index: 9, Name: floppytop2, message: lol, toxic:['Not Toxic']
Index: 10, Name: kard_ttv, message: super mario world is perfect, toxic:['Not Toxic']
Index: 11, Name: darkshadowx91, message: Wind waker KEKW, toxic:['Toxic']
Index: 12

In [17]:
import vowpal_wabbit_next as vw
import random
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import json
from collections import defaultdict  

from typing import List, Tuple

In [18]:
toxicity = pd.read_json("social_media_toxicity_dataset.json")
toxicity.head()

,worker_id,task_id,task_response_id,text,Is this text toxic?
0,4RHRV9MACQGW,ccd0a1da-0e6a-453e-a6d1-6418cc4a5546,3d66075c-623e-4e64-93ac-123d379d28f0,I came here to say this exactly!,Not Toxic
1,R7XYZ4FKMFF9,d977c704-4ae3-4381-99de-95014ee57a05,acbd5847-698e-4098-9499-d547948a2da2,Thank you :),Not Toxic
2,MNTYRY6PYPQF,89a4c206-dfbc-40e8-ac73-9bb1b5c8e56e,2def7071-3e27-4b63-bd66-99f041363e2a,I feel a subreddit being born,Not Toxic
3,DR6XNZMT9KRH,41add9eb-bbe3-47d9-8ea2-b571f96fe655,b6e11a16-686d-43da-87f4-8201d8bb2238,Disturbing wholesomeness should be a thing,Not Toxic
4,G6VJRCCGZ9D6,883f5bd6-058c-460f-8e06-c0793af441d1,7f0bdd8f-428a-4ecb-b432-8e6b1f3706c0,Smite jinx... int or pentakill? Both.,Not Toxic


In [19]:
for index, row in toxicity.iterrows():
    if row["Is this text toxic?"] == "Not Toxic":
        text = row["text"]
        print(f"0, {text}")
    elif row["Is this text toxic?"] == "Toxic":
        text = row["text"]
        print(f"1, {text}")

0, I came here to say this exactly!
0, Thank you :)
0, I feel a subreddit being born
0, Disturbing wholesomeness should be a thing
0, Smite jinx... int or pentakill? Both.
0, Girl I feel you on those scares! Can’t wait to also get my tubes tied so I can live out the days before my period without anxiety :(
0, Please, it's the funniest timeline and i need it to happen
0, I was a little disappointed by the lack of a turkey offer this year, but honestly it’s awesome they do something like this at all. Looking forward to the upcoming Walmart visit!
0, Hello, I am in the same boat. If you would like to pm me we can swap and try to help. I feel like mine is too essay like and robotic.
0, Stringy Mc String-face
0, Waiting for Hadestown announcements....
0, when my cat calls it a night and curls up next to be in bed I can literally feel my stress level go down. don't know what it is but its great, would probably be the same with a dog.
1, you're a stupid pos
0, Coronavirus is the one and only 

In [20]:
toxicity1 = pd.read_json("social_media_toxicity_dataset.json")

# This function modifies (context, action, cost, probability) to VW friendly format
def convert_training_data_to_vw_format(data, context, cb_label=None, actions=None):
    # if cb_label is not None:
    #     chosen_action, cost, prob = cb_label
    example_string = ""
    
    data = toxicity
    for index, row in data.iterrows():
        if row["Is this text toxic?"] == "Not Toxic":
            text = row["text"]
            example_string += "0 | ".format(text)
            
        elif row["Is this text toxic?"] == "Toxic":
            text = row["text"]
            example_string += "1 | ".format(text)
        
        example_string += "Chat {}\n".format(
            row["text"]
        )
    # print(example_string[:-1])
    
    # for action in actions:
    #     if cb_label is not None and action == chosen_action:
    #         example_string += "0:{}:{} ".format(cost, prob)
    #     example_string += "|Action article={} \n".format(action)
    # Strip the last newline
    return example_string
    # return example_string
    


In [21]:
convert_training_data_to_vw_format(toxicity1, "Chat", cb_label=None)

'0 | Chat I came here to say this exactly!\n0 | Chat Thank you :)\n0 | Chat I feel a subreddit being born\n0 | Chat Disturbing wholesomeness should be a thing\n0 | Chat Smite jinx... int or pentakill? Both.\n0 | Chat Girl I feel you on those scares! Can’t wait to also get my tubes tied so I can live out the days before my period without anxiety :(\n0 | Chat Please, it\'s the funniest timeline and i need it to happen\n0 | Chat I was a little disappointed by the lack of a turkey offer this year, but honestly it’s awesome they do something like this at all. Looking forward to the upcoming Walmart visit!\n0 | Chat Hello, I am in the same boat. If you would like to pm me we can swap and try to help. I feel like mine is too essay like and robotic.\n0 | Chat Stringy Mc String-face\n0 | Chat Waiting for Hadestown announcements....\n0 | Chat when my cat calls it a night and curls up next to be in bed I can literally feel my stress level go down. don\'t know what it is but its great, would proba

In [22]:
def parse_lines(parser: vw.TextFormatParser, input_str: str) -> List[vw.Example]:
    return [parser.parse_line(line) for line in input_str.split("\n")]

In [23]:
def sample_custom_pmf(pmf: List[Tuple[int, float]]) -> Tuple[int, float]:
    values = [x for (_, x) in pmf]
    total = sum(values)
    scale = 1 / total
    values = [x * scale for x in values]
    draw = random.random()
    sum_prob = 0.0
    for index, prob in enumerate(values):
        sum_prob += prob
        if sum_prob > draw:
            return pmf[index][0], prob

In [24]:
def get_action(workspace: vw.Workspace, parser: vw.TextFormatParser, context, actions):
    pmf = workspace.predict_one(
        parse_lines(parser, convert_training_data_to_vw_format(context, actions))
    )
    chosen_action_index, prob = sample_custom_pmf(pmf)
    return actions[chosen_action_index], prob

In [25]:

# # display preference matrix
# def get_preference_matrix(cost_fun):
#     def expand_grid(data_dict):
#         rows = itertools.product(*data_dict.values())
#         return pd.DataFrame.from_records(rows, columns=data_dict.keys())

#     df = expand_grid({"users": users, "times_of_day": times_of_day, "actions": actions})
#     df["cost"] = df.apply(
#         lambda r: cost_fun({"user": r[0], "time_of_day": r[1]}, r[2]), axis=1
#     )

#     return df.pivot_table(
#         index=["users", "times_of_day"], columns="actions", values="cost"
#     )


# get_preference_matrix(get_cost)

In [67]:
import re

def preprocess_message(message):
    # 1. Lowercasing
    message = message.lower()

    # 2. Removing Punctuation
    message = re.sub(r'[^\w\s]', '', message)

    # 3. Tokenization (simple split-based tokenization)
    tokens = message.split()

    # 4. Handling URLs and Usernames
    tokens = ['[URL]' if re.match(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', word) else word for word in tokens]
    tokens = ['@user' if word.startswith('@') else word for word in tokens]

    # 5. Handling Emotes and Platform-specific Slang (here we only handle 'Kappa', you can extend this)
    tokens = ['[EMOTE]' if word == 'kappa' else word for word in tokens]

    # 6. Formatting for VW will be done outside this function

    # 7. Handling Missing Data is also done outside this function 

    # 8. Stemming (simple stemming approach)
    def simple_stemmer(word):
        suffixes = ["ing", "ly", "ed", "ious", "ies", "ive", "es", "s", "ment"]
        for suffix in suffixes:
            if word.endswith(suffix):
                return word[:-len(suffix)]
        return word

    stemmed_tokens = [simple_stemmer(word) for word in tokens]

    # Rejoin tokens into a string
    preprocessed_message = ' '.join(stemmed_tokens)

    return preprocessed_message

# if __name__ == "__main__":
#     sample_message = "@Streamer123 This Streamer is AWESOME! Check out https://www.example.com. I'm just joking Kappa"
#     print(preprocess_message(sample_message))


streamer123 thi streamer i awesome check out httpswwwexamplecom im just jok [EMOTE]


In [68]:
import itertools
import random
from typing import List, Tuple

# 1. Helper functions:

def get_vw_example_from_row(row):
    """Converts a row of the toxicity dataset into VW format."""
    label = '1' if row["Is this text toxic?"] == "Toxic" else '-1'
    return "{} | Chat {}\n".format(label, preprocess_message(row["text"]))

# 2. Refactor your data transformation function:

def convert_training_data_to_vw_format(data):
    """Converts the entire toxicity dataset into VW format."""
    return ''.join([get_vw_example_from_row(row) for _, row in data.iterrows()])

# 3. Define the core functions to make predictions using VW:

def parse_lines(parser: vw.TextFormatParser, input_str: str) -> List[vw.Example]:
    result = []
    for line in input_str.split("\n"):
        # print(line)
        # if line.strip() != "":
        result.append(line)
    return result
                
# def get_action(workspace: vw.Workspace, parser: vw.TextFormatParser, context, actions):
#     # print(parse_lines(parser, convert_training_data_to_vw_format(context)))

#     pmf = workspace.predict_one(
#         parse_lines(parser, convert_training_data_to_vw_format(context))
#     )
#     chosen_action_index, prob = sample_custom_pmf(pmf)
#     return actions[chosen_action_index], prob

def get_vw_commands(workspace: vw.Workspace, parser: vw.TextFormatParser, context):
    # parse_lines(parser, convert_training_data_to_vw_format(context))
    return parse_lines(parser, convert_training_data_to_vw_format(context))
    


In [69]:
toxicity = pd.read_json("social_media_toxicity_dataset.json")


In [70]:
workspace = vw.Workspace()
parser = vw.TextFormatParser
# print(get_action(workspace, parser, toxicity, ["Toxic", "Not Toxic"]))
parse_lines(parser, convert_training_data_to_vw_format(toxicity))

['-1 | i came here to say thi exact',
 '-1 | thank you',
 '-1 | i feel a subreddit be born',
 '-1 | disturb wholesomenes should be a th',
 '-1 | smite jinx int or pentakill both',
 '-1 | girl i feel you on those scar cant wait to also get my tub ti so i can l out the day before my period without anxiety',
 '-1 | please it the funniest timeline and i ne it to happen',
 '-1 | i wa a little disappoint by the lack of a turkey offer thi year but honest it awesome they do someth like thi at all look forward to the upcom walmart visit',
 '-1 | hello i am in the same boat if you would like to pm me we can swap and try to help i feel like mine i too essay like and robotic',
 '-1 | stringy mc stringface',
 '-1 | wait for hadestown announcement',
 '-1 | when my cat call it a night and curl up next to be in b i can literal feel my stres level go down dont know what it i but it great would probab be the same with a dog',
 '1 | youre a stupid po',
 '-1 | coronaviru i the one and on reason trump lost

In [72]:
# write to file all the commands
with open("training_data.txt", "w") as f:
    for command in parse_lines(parser, convert_training_data_to_vw_format(toxicity)):
        f.write(command + "\n")
        

In [63]:
# # Train the model
# !vw --cb_adf -d training_data_vw.txt -f cb_model.vw --quiet

In [57]:
# # Test the model
# !vw -i cb_model.vw -t -d testing_vw.txt -p cb_predictions.txt --quiet